In [1]:
import cv2
from cnn import CNN
import tensorflow as tf
cnn = CNN()

In [2]:
data_list = [
    {"folder":"image_t_face","label":"0"},
    {"folder":"image_f_face","label":"1"}
]
    
    
test_list = [
    {"folder":"img_t_face_test", "label":"0"}
    #{"folder":"img_f_face_test", "label":"1"}
]

train_image, train_label, _ = cnn.read_image_cv(data_list)
test_image , test_label , test_image_list = cnn.read_image_cv(test_list)  

In [5]:
import os
log_dir = os.getcwd() + cnn.LOG_DIR

In [6]:
with tf.Graph().as_default():
        
    images_placeholder = tf.placeholder("float", shape=(None, cnn.IMAGE_PIXELS))
    labels_placeholder = tf.placeholder("float", shape=(None, cnn.NUM_CLASSES))
    keep_prob = tf.placeholder("float")

    
    softmax = cnn.inference(images_placeholder, keep_prob)
    loss_value = cnn.loss(softmax, labels_placeholder)
    train_op = cnn.training(loss_value, cnn.LEARNING_RATE)
    acc = cnn.accuracy(softmax, labels_placeholder)
    
    saver = tf.train.Saver()

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
            
    summary_op = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
        
        
    for step in range(cnn.MAX_STEPS):
        for i in range(len(train_image)/cnn.BATCH_SIZE):
            batch = cnn.BATCH_SIZE*i
            sess.run(train_op, feed_dict={
                    images_placeholder: train_image[batch:batch+cnn.BATCH_SIZE],
                    labels_placeholder: train_label[batch:batch+cnn.BATCH_SIZE],
                    keep_prob: 0.5})

        #train_accuracy = sess.run(acc, feed_dict={
        #        images_placeholder: train_image,
        #        labels_placeholder: train_label,
        #        keep_prob: 1.0})
        #print "step %d, training accuracy %g"%(step, train_accuracy)

        #p = sess.run(softmax,  feed_dict={
        #     images_placeholder: train_image,
        #    keep_prob: 1.0})
        #print "step %d, p %s"%(step, str(p))
        
        train_loss = sess.run(loss_value,  feed_dict={
            images_placeholder: train_image,
            labels_placeholder: train_label,
            keep_prob: 1.0})
        print "step %d, xentropy %s"%(step, str(train_loss))
            
        if step % 10 == 0 and step > 0 and train_loss < 5.0:
            cnn.display_test_prob(sess, softmax, test_image, "obama", images_placeholder,keep_prob)

        summary_str = sess.run(summary_op, feed_dict={
            images_placeholder: train_image,
            labels_placeholder: train_label,
            keep_prob: 1.0})
        summary_writer.add_summary(summary_str, step)

  
    print "test accuracy %g"%sess.run(acc, feed_dict={
        images_placeholder: test_image,
        labels_placeholder: test_label,
        keep_prob: 1.0})
    

    save_path = saver.save(sess, "model.ckpt")

step 0, p [[  8.63580864e-24   1.00000000e+00]
 [  6.53631456e-21   1.00000000e+00]
 [  1.12660046e-24   1.00000000e+00]
 ..., 
 [  3.56735759e-20   1.00000000e+00]
 [  7.04531983e-27   1.00000000e+00]
 [  4.12409234e-25   1.00000000e+00]]
step 0, xentropy 13332.6
step 1, p [[  1.26285577e-05   9.99987364e-01]
 [  3.62092360e-05   9.99963760e-01]
 [  2.62250887e-06   9.99997377e-01]
 ..., 
 [  2.16517174e-06   9.99997854e-01]
 [  1.79636777e-06   9.99998212e-01]
 [  1.44730416e-06   9.99998569e-01]]
step 1, xentropy 2893.11
step 2, p [[  5.72943836e-02   9.42705572e-01]
 [  2.31984630e-02   9.76801455e-01]
 [  1.01010893e-02   9.89898860e-01]
 ..., 
 [  3.18915257e-03   9.96810853e-01]
 [  1.49605516e-02   9.85039473e-01]
 [  1.38193864e-04   9.99861836e-01]]
step 2, xentropy 972.72
step 3, p [[  6.20708823e-01   3.79291177e-01]
 [  2.17803210e-01   7.82196820e-01]
 [  1.50952369e-01   8.49047661e-01]
 ..., 
 [  1.97569188e-02   9.80243146e-01]
 [  2.11718217e-01   7.88281798e-01]
 [  

InvalidArgumentError: Nan in summary histogram for: conv1/W_conv1
	 [[Node: conv1/W_conv1 = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](conv1/W_conv1/tag, conv1/Variable/read)]]

Caused by op u'conv1/W_conv1', defined at:
  File "//anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "//anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "//anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "//anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "//anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "//anaconda/lib/python2

In [7]:
import os
import getface as gf
cwd = os.getcwd()
image_path = cwd + "/image_t"
files = os.listdir(image_path)
len(files)

313

In [10]:
for f in files:
    dist_path = cwd+"/image_t_face"
    image_name = f
    r = gf.cutout_face(image_path, image_name, dist_path)
    print r

{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'error', 'code': 1, 'http_status': 400}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'error', 'code': 1, 'http_status': 400}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 2}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type'

In [9]:
dist_path = cwd+"/image_t_face"
len(os.listdir(dist_path))

255

In [12]:
import os
import getface as gf
cwd = os.getcwd()
image_path = cwd + "/image"
files = os.listdir(image_path)
len(files)

1001

In [13]:
for f in files:
    dist_path = cwd+"/image_f_face"
    image_name = f
    r = gf.cutout_face(image_path, image_name, dist_path)
    print r

{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 4}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'error', 'code': 1, 'http_status': 400}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'error', 'code': 1, 'http_status': 400}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type'